In [1]:
#imports    
import folium
from folium import plugins

In [2]:
#initializing GEE - try/except block
try:
    import ee
except ModuleNotFoundError: 
    !pip install earth-engine-api
    import ee

In [3]:
#start the authentication - try/except block
try:
    ee.Initialize()
except ee.EEException:
    !earthengine authenticate

In [4]:
#localization sample using ee.Geometry
coordinates_sample = ee.Geometry.Point([-52, -3])

In [5]:
#nicfi basemap ImageCollectin to Americas
#you need to configure your access here: https://developers.planet.com/docs/integrations/gee/nicfi/
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas').filterBounds(coordinates_sample)

In [6]:
#NDVI to Planet NICFI Basemaps
def ndvi_nicfibasemaps(image):
    ndvi = image.normalizedDifference(['N', 'R']).rename('NDVI')
    return image.addBands(ndvi)

#Visualization with Folium Map GEE integrated
def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            fill = False,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    except:
        print("Could not display {}".format(name))

#Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [7]:
#filter basemaps by date and get the first image from filtered results - map NDVI 
basemap_jun2021_NDVI = nicfi.filter(ee.Filter.date('2021-06-01','2021-06-30')).map(ndvi_nicfibasemaps).first();

In [8]:
#filter basemaps by date and get the first image from filtered results - RGB
basemap_jun2021 = nicfi.filter(ee.Filter.date('2021-06-01','2021-06-30')).first();

In [9]:
# Create a folium map object.
Map = folium.Map(location=[-3, -52], zoom_start=15)

# Set NDVI visualization parameters.
nicfibasemap_vis_ndvi = {'min': 0.13, 'max': 0.94, 'palette': ['red', 'orange', 'yellow', 'green']}
# Set RGB visualization parameters.
nicfibasemap_vis ={'bands': ['R', 'G', 'B'], 'min': 64, 'max': 5454, 'gamma':1.8}

#  Add RGB NICFI Basemap
Map.add_ee_layer(basemap_jun2021, nicfibasemap_vis, 'NICFI Basemap RGB ')

# Add NDVI NICFI Basemap
Map.add_ee_layer(basemap_jun2021_NDVI.select("NDVI"), nicfibasemap_vis_ndvi, 'NICFI Basemap NDVI')

# Add a layer control panel to the map.
Map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(Map)

# Create title text
loc = 'NDVI/RGB NICFI Basemaps - June 2021'
# Configure title HTML
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)
# Set title element
Map.get_root().html.add_child(folium.Element(title_html))
Map